# Stage directions: basic features

This notebook contains:

* directions extraction
* basic values calculation

## Imports
<span style="font-size:larger;color:blue;">Run this before everything else!</span>

In [1]:
# imports
import os
from lxml import etree
import re
import pandas as pd

# we need this namespaces parameter to search in TEI-encoded files correctly
tei_ns = {'tei': 'http://www.tei-c.org/ns/1.0'}

## Preparation

### Data
I used [RusDraCor](https://github.com/dracor-org/rusdracor) as of 8th February, 2018: 80 plays, dating from 18th to 20th century. More precised data on the corpus is below.

In [2]:
corpus_path = ".." + os.sep + "RusDraCor"

### Play class
I want to create a table with information on the plays we research, so it will be much easier to have a separate ```Play``` class with all the data.

In [3]:
class Play:
    def __init__(self, path, author, title, year_written, year_published, amount_of_directions, amount_of_acts):
        self.path = path
        self.author = author
        self.title = title
        self.year_written = year_written
        self.year_published = year_published
        self.amount_of_directions = amount_of_directions
        self.amount_of_acts = amount_of_acts

## Extraction

### Play information
The majority of information pieces can be extracted from the ```html <teiHeader></teiHeader>``` in XML itself. Each act is wrapped in ```<div type="act">```, so counting the acts also isn't much of a problem.

In [4]:
def get_play_data(play_path):
    reg_space = re.compile("\s{2,}")
    root = etree.parse(play_path)
    title = root.find(".//tei:titleStmt/tei:title", tei_ns).text
    title = reg_space.sub(" ", title)
    author = root.find(".//tei:titleStmt/tei:author", tei_ns).text
    try:
        written_str = root.find(".//tei:sourceDesc/tei:bibl/tei:bibl/tei:date[@type=\"written\"]", tei_ns).attrib["when"]
        year_written = int(written_str)
    except:
        year_written = -1
    try:
        published_str = root.find(".//tei:sourceDesc/tei:bibl/tei:bibl/tei:date[@type=\"print\"]", tei_ns).attrib["when"]
        year_published = int(published_str)
    except:
        year_published = -1
    acts = len(root.findall(".//tei:text/tei:body/tei:div[@type=\"act\"]", tei_ns))
    return author, title, year_written, year_published, acts    

### Directions
Basically, all the directions are wrapped by the ```<stage></stage>``` tag, so we can just use regex. The only problem with this approach is that sometimes directions are on separate lines, and many spaces and tabs may occur in between two words — and this can also be fixed with the help of regex. 

In [5]:
def get_directions(play_path):
    reg_space = re.compile("\s{2,}")
    root = etree.parse(play_path)
    directions_tags = root.findall(".//tei:stage", tei_ns)
    directions_text = []
    for direction in directions_tags:
        try:
            text = direction.text
            text = reg_space.sub(" ", text)
            text = text.replace("\xa0", " ")
            text = text.strip("\(\) \.")
            text = text.lower()
            directions_text.append(text)
        except:
            pass
    return directions_text

## Putting it all together

### Saving the data
The following is being done in order to save the extraced directions just in case (and not to extract them from the play every time we need them).

In [6]:
# some basic stuff we need in order to save everything properly
directions_path = ".." + os.sep + "data"
if not os.path.exists(directions_path):
    os.mkdirs(directions_path)

all_directions_file = directions_path + os.sep + "all_directions.txt"
if not os.path.exists(all_directions_file):
    f = open(all_directions_file, "w", encoding="utf-8")
    f.close()

In [7]:
def save_directions(play_name, directions, option):
    if option == "sep":
        directions_file = directions_path + os.sep + play_name.replace("xml", "txt")
        with open(directions_file, "w", encoding="utf-8") as directions_save:
            directions_save.write("\n".join(directions))
    elif option == "all":
        with open(all_directions_file, "a", encoding="utf-8") as all_directions_save:
            all_directions_save.write("\n".join(directions) + "\n")

### Working with the corpus

Now, we traverse all the plays in the ```corpus_folder``` and do the following: 
* extract all the directions and save them: 
    * in a ```directions_list``` variable,
    * in a separate file,
    * in a file that contains all the directions
* extract information about a play and save it into ```play_list``` (to convert it into a Pandas dataframe later)

In [8]:
# this is required because I'm using a Mac, which sometimes creates system folders like .DS_Store;
# in any other cases -- never mind
play_files = [item for item in os.listdir(corpus_path) if item.endswith(".xml")]

In [9]:
directions_list = []
play_list = []

for play in play_files:
    play_path = corpus_path + os.sep + play
    
    st_dirs = get_directions(play_path)
    directions_list.append(st_dirs)
    save_directions(play, st_dirs, option="separately")
    save_directions(play, st_dirs, option="all")
    
    author, title, year_written, year_published, acts = get_play_data(play_path)
    play = {"Path": play_path, 
            "Author": author, 
            "Title": title, 
            "Written": year_written, 
            "Published": year_published, 
            "Amount of directions": len(st_dirs), 
            "Amount of acts": acts}
    play_list.append(play)

In [10]:
play_df = pd.DataFrame(play_list)
play_df

,Amount of acts,Amount of directions,Author,Path,Published,Title,Written
0,0,38,"Пушкин, Александр Сергеевич",../RusDraCor/pushkin-stseny-iz-rytsarskih-vrem...,1837,Сцены из рыцарских времен,1835
1,0,687,"Тургенев, Иван Сергеевич",../RusDraCor/turgenev-holostjak.xml,1849,Холостяк,1849
2,2,254,"Гоголь, Николай Васильевич",../RusDraCor/gogol-zhenitba.xml,1842,Женитьба,1835
3,3,88,"Крылов, Иван Андреевич",../RusDraCor/krylov-sonnyj-poroshok-ili-pohisc...,1905,Сонный порошок или похищенная крестьянка,1798
4,0,80,"Блок, Александр Александрович",../RusDraCor/blok-neznakomka.xml,1907,Незнакомка,1906
5,0,35,"Гоголь, Николай Васильевич",../RusDraCor/gogol-lakeiskaja.xml,1842,Лакейская,1840
6,4,372,"Булгаков, Михаил Афанасьевич",../RusDraCor/bulgakov-dni-turbinyh.xml,1955,Дни Турбиных,1926
7,0,9,"Пушкин, Александр Сергеевич",../RusDraCor/pushkin-pir-vo-vremja-chumy.xml,1832,Пир во время чумы,1830
8,3,675,"Сухово-Кобылин, Александр Васильевич",../RusDraCor/sukhovo-kobylin-smert-tarelkina.xml,1869,Смерть Тарелкина,-1
9,0,66,"Гумилёв, Николай Степанович",../RusDraCor/gumilyov-ditja-allaha.xml,1917,Дитя Аллаха,1916
